In [3]:
import os

train_done = True
study_id = 'original'
data_id = 'pan'

size = '100p'
per = 'all'

data_ = 'TransMut'
encoding_ = 'blosum'

#id_ = 'OF_TData_pad14_{}_55ep_onlyBlosum'.format(size)
id_ = 'OF_TData_pad14_{}_onlyBlosum_hlac_NOPOS'.format(size)

data_dir = '/home/s202357/thesis/transmut/data/transmut_github/'
model_folder = '/home/s202357/thesis/transmut/pipeline/{}/models/'.format(study_id)

result_folder = '/home/s202357/thesis/transmut/pipeline/{}/results/'.format(study_id)

plot_name = result_folder + id_+ '_plot_'
df_name = result_folder + id_+ '_df_'

if not os.path.exists(model_folder):
        print('creating dirs')
        os.makedirs(model_folder)
        os.makedirs(result_folder)
else:
        print('dirs exist')


dirs exist


In [4]:
!gpustat

mnemosyne            Tue Jun  7 08:07:04 2022  470.103.01
[0] NVIDIA TITAN V   | 32'C,   0 % |     0 / 12066 MB |
[1] NVIDIA TITAN V   | 37'C,   0 % |     0 / 12066 MB |
[2] NVIDIA TITAN V   | 37'C,   0 % |     0 / 12066 MB |
[3] NVIDIA TITAN V   | 36'C,   0 % |     0 / 12066 MB |
[4] NVIDIA TITAN V   | 35'C,   0 % |     0 / 12066 MB |
[5] NVIDIA TITAN V   | 32'C,   0 % |  3210 / 12066 MB | ansup(1591M) ansup(1615M)
[6] NVIDIA TITAN V   | 34'C,   0 % |     0 / 12066 MB |


In [5]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [6]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
#from sklearn.metrics import classification_report
#from sklearn.utils import class_weight

import sys
import gc
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

#import framework as fr

In [7]:
sys.path.insert(0, '/home/s202357/thesis/transmut/source/')

import blosum as enc

In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    use_cuda = True
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) avalable.


In [9]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [10]:
blosum_dict = {
'A' : [5,-2,-1,-2,-1,-1,-1,0,-2,-1,-2,-1,-1,-3,-1,1,0,-3,-2,0],
'R' : [-2,7,-1,-2,-4,1,0,-3,0,-4,-3,3,-2,-3,-3,-1,-1,-3,-1,-3],
'N' : [-1,-1,7,2,-2,0,0,0,1,-3,-4,0,-2,-4,-2,1,0,-4,-2,-3],
'D' : [-2,-2,2,8,-4,0,2,-1,-1,-4,-4,-1,-4,-5,-1,0,-1,-5,-3,-4],
'C' : [-1,-4,-2,-4,13,-3,-3,-3,-3,-2,-2,-3,-2,-2,-4,-1,-1,-5,-3,-1],
'Q' : [-1,1,0,0,-3,7,2,-2,1,-3,-2,2,0,-4,-1,0,-1,-1,-1,-3],
'E' : [-1,0,0,2,-3,2,6,-3,0,-4,-3,1,-2,-3,-1,-1,-1,-3,-2,-3],
'G' : [0,-3,0,-1,-3,-2,-3,8,-2,-4,-4,-2,-3,-4,-2,0,-2,-3,-3,-4],
'H' : [-2,0,1,-1,-3,1,0,-2,10,-4,-3,0,-1,-1,-2,-1,-2,-3,2,-4],
'I' : [-1,-4,-3,-4,-2,-3,-4,-4,-4,5,2,-3,2,0,-3,-3,-1,-3,-1,4],
'L' : [-2,-3,-4,-4,-2,-2,-3,-4,-3,2,5,-3,3,1,-4,-3,-1,-2,-1,1],
'K' : [-1,3,0,-1,-3,2,1,-2,0,-3,-3,6,-2,-4,-1,0,-1,-3,-2,-3],
'M' : [-1,-2,-2,-4,-2,0,-2,-3,-1,2,3,-2,7,0,-3,-2,-1,-1,0,1],
'F' : [-3,-3,-4,-5,-2,-4,-3,-4,-1,0,1,-4,0,8,-4,-3,-2,1,4,-1],
'P' : [-1,-3,-2,-1,-4,-1,-1,-2,-2,-3,-4,-1,-3,-4,10,-1,-1,-4,-3,-3],
'S' : [1,-1,1,0,-1,0,-1,0,-1,-3,-3,0,-2,-3,-1,5,2,-4,-2,-2],
'T' : [0,-1,0,-1,-1,-1,-1,-2,-2,-1,-1,-1,-1,-2,-1,2,5,-3,-2,0],
'W' : [-3,-3,-4,-5,-5,-1,-3,-3,-3,-3,-2,-3,-1,1,-4,-4,-3,15,2,-3],
'Y' : [-2,-1,-2,-3,-3,-1,-2,-3,2,-1,-1,-2,0,4,-3,-2,-2,2,8,-1],
'V' : [0,-3,-3,-4,-1,-3,-3,-4,-4,4,1,-3,1,-1,-3,-2,0,-3,-1,5]
}

for key, val in zip(blosum_dict.keys(), blosum_dict.values()):
    print(key, len(val))

A 20
R 20
N 20
D 20
C 20
Q 20
E 20
G 20
H 20
I 20
L 20
K 20
M 20
F 20
P 20
S 20
T 20
W 20
Y 20
V 20


In [11]:
def pep_to_blosum(pep):
    return [blosum_dict[word] for word in pep]

In [12]:
def best_treshold(df_results):
    acc_best = -999
    best_i = -999

    for i in np.arange(0, 1, 0.1):
        df_results['pred_binary'] = np.where((df_results.pred > i) , 1, 0)
        acc = accuracy_score(df_results['target'], df_results['pred_binary'])
        if acc > acc_best:
            df = df_results
            best_i = i
            acc_best = acc
            
    return acc_best, best_i, df


def best_treshold(df_results, t= []):
    acc_best = -999
    best_i = -999

    if t == []:
        for i in np.arange(0, 1, 0.1):
            df_results['pred_binary'] = np.where((df_results.pred > i) , 1, 0)
            acc = accuracy_score(df_results['target'], df_results['pred_binary'])
            if acc > acc_best:
                df = df_results
                best_i = i
                acc_best = acc
    else:
        best_i = t[0]
        df_results['pred_binary'] = np.where((df_results.pred > t[0]) , 1, 0)
        df = df_results
    return acc_best, best_i, df


def pkl(id_, n_layers, n_heads, model_num, d_model):
    return '{}_d{}_layer{}_multihead{}_MODEL{}.pkl'.format(id_, d_model, n_layers, n_heads, model_num)


def make_data_pep_hla(data):
    pep_inputs, hla_inputs, labels = [], [], []
    cc = 0
    tt = time.time()
    
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        
        #if cc % 100000 == 0:
        #    print("Blosum completed:", cc, end = " ")
        #    print(", Time:", round((time.time()-tt)/60, 2), "mins.")
            
        bl = pep_to_blosum(pep)
        max_len=14
        pep_len = len(pep)
        for t in range(max_len-pep_len):
            xs = [-1]*20
            bl.append(xs)
            
        pep_inputs.append(bl)
        bl_hla = pep_to_blosum(hla) 
        hla_inputs.append(bl_hla)

        cc += 1
    
    return torch.tensor(pep_inputs, dtype=torch.float).to(device), torch.tensor(hla_inputs, dtype=torch.float)


class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self):
        l = self.pep_inputs.shape[0]
        return l

    def __getitem__(self, idx):
        try:
            p = self.pep_inputs[idx]
            h = self.hla_inputs[idx]
            lb = self.labels[idx]
        except:
            print("MyDataSet")
            print("shape pep_inputs", np.shape(self.pep_inputs))
            print("idx:", idx)
            #print("shape np.shape(pep_inputs[idx]) -> ", np.shape(self.pep_inputs[idx]))
            print(pep_inputs.shape[0])
            print(self.l)
        return p, h, lb

In [14]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q, _ = seq_q.size()
    batch_size, len_k, _ = seq_k.size()
    
    seq_q = torch.empty(batch_size, len_q).to(device)
    seq_k = torch.empty(batch_size, len_k).to(device)
    
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn


class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        
        residual, batch_size = input_Q, input_Q.size(0)

        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]
        
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        
        return nn.LayerNorm(d_model).to(device)(output + residual), attn

    
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        #self.src_emb = nn.Embedding(vocab_size, d_model)
        #self.pos_emb = PositionalEncoding(d_model)
        #self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        ##B## enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        #enc_outputs = self.pos_emb(enc_inputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_outputs = enc_inputs
        return enc_outputs, 0

    
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        #self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        #print("\n\nAfter concat:", enc_outputs, "\nsize", np.shape(enc_outputs))
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        #dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = enc_outputs.view(enc_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        #print("len(dec_outputs), len(dec_outputs[0])", len(dec_outputs), len(dec_outputs[0]))
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), 0, 0, 0

In [15]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)



def performances_QM_(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)



def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])



f_mean = lambda l: sum(l)/len(l)



def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [16]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list, dec_attns_train_list = [], []
    tr = 0
    for train_pep_inputs, train_hla_inputs, train_labels in train_loader:
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        tr += 1
        train_pep_inputs, train_hla_inputs, train_labels = train_pep_inputs.to(device), train_hla_inputs.to(device), train_labels.to(device)

        t1 = time.time()
        train_outputs, _, _, train_dec_self_attns = model(train_pep_inputs, train_hla_inputs)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss)
#         dec_attns_train_list.append(train_dec_self_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, dec_attns_train_list


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

In [17]:
pep_max_len = 14 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load( data_dir + 'Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

# Transformer Parameters
d_model = 20  # Embedding Size
d_k = d_v = 20  # dimension of K(=Q), V

d_ff = 512 # FeedForward dimension
n_layers = 1  # number of Encoder of Decoder Layer
n_heads = 3

batch_size = 1024
epochs = 25
threshold = 0.5

In [18]:
cv = 5
count = 0
index_order = []

for outer in range(cv):
        #print(f"\nk_outer: {outer}")
        #print("test: " + str(outer))
        test_idx = outer
        for inner in range(cv):
            #print(f"\t\nk_inner: {inner}")
            if inner!=outer:
                #print("\tval: " + str(inner))
                val_idx = inner
                train_index = list()
                for t in range(5):
                    if t!=inner and t!=outer:
                        train_index.append(t)
                #print("\ttrain:", str(train_index))
                count += 1  
                index_order.append([test_idx, val_idx, train_index])
                
index_order_nested = index_order


In [19]:
def data_with_loader(index_order_idx, type_ ='train', fold = None,  batch_size = 1024):
        
    if type_ == 'train':
        print("prepare training data:")
        df_list_pep = []
        df_list_hla = []
        df_list_labels = []
            
        idx_0 = index_order_nested[index_order_idx][2][0]
        idx_1 = index_order_nested[index_order_idx][2][1]
        idx_2 = index_order_nested[index_order_idx][2][2]
        
        for i in [idx_0, idx_1, idx_2]:
            data = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0)
            pep_inputs, hla_inputs = make_data_pep_hla(data)
            labels = torch.LongTensor( [x for x in data.label]).to(device)
            print("make data {} done".format(i))
            df_list_pep.append(pep_inputs)
            df_list_hla.append(hla_inputs)
            df_list_labels.append(labels)     
        
        print("make data done")
        hla_inputs = torch.cat( (df_list_hla[0], df_list_hla[1], df_list_hla[2]) , 0)
        print("hla done")
        del df_list_hla
        gc.collect()
        torch.cuda.empty_cache()
        
        pep_inputs = torch.cat( (df_list_pep[0], df_list_pep[1], df_list_pep[2]) , 0)
        print("pep done")
        del df_list_pep
        gc.collect()
        torch.cuda.empty_cache()
        
        labels = torch.cat( (df_list_labels[0], df_list_labels[1], df_list_labels[2]) , 0)
        print("labels done")
        del df_list_labels
        gc.collect()
        torch.cuda.empty_cache()
        
        print("TRAIN ID:", [idx_0, idx_1, idx_2], len(pep_inputs))
        print("prepare training data done.")
        
    elif type_ == 'test':
        print("prepare test data:")
        idx = index_order_nested[index_order_idx][0]
        data = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(idx), index_col = 0)
        pep_inputs, hla_inputs = make_data_pep_hla(data)
        labels = torch.LongTensor( [x for x in data.label] ).to(device)
        print("TEST ID:", idx, len(pep_inputs))
        print("prepare test data done.")
        
    elif type_ == 'val':
        print("prepare val data:")
        idx = index_order_nested[index_order_idx][1]
        data = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(idx), index_col = 0)
        pep_inputs, hla_inputs = make_data_pep_hla(data)
        labels = torch.LongTensor( [x for x in data.label] ).to(device)
        print("VAL ID:", idx, len(pep_inputs))
        print("prepare val data done.")
        
    print("shape pep inputs:", np.shape(pep_inputs))
    print("shape hla inputs:", np.shape(hla_inputs))
    print("shape labels:", np.shape(labels))
    
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    return pep_inputs, hla_inputs, labels, loader


In [21]:
fold_avg_list = [list(range(0,4)), list(range(4,8)), list(range(8,12)), list(range(12,16)), list(range(16,20))] 

In [22]:
ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list, test_fold_metrics_list = [], [], []
attns_train_fold_dict, attns_val_fold_dict = {}, {}
loss_train_fold_dict, loss_val_fold_dict, = {}, {}
metrics_all_val = [] 
metrics_all_train = []

time_train_c = time.time()
fold_avg_index = -1
for fold in tqdm(range(16,20,1)):
    
        test_bool = False
        #if fold % 4 == 0 and fold != 0:
        #    test_bool = True
        #    fold_avg_index += 1
            
        print('=====Inner Fold-{}====='.format(fold))
        print('-----Generate data loader-----')
        index_order_idx = fold
        print(index_order_idx)
        train_pep_inputs, train_hla_inputs, train_labels, train_loader = data_with_loader(index_order_idx, type_ = 'train', fold = fold,  batch_size = batch_size)
        val_pep_inputs, val_hla_inputs, val_labels, val_loader = data_with_loader(index_order_idx, type_ = 'val', fold = fold,  batch_size = batch_size)

        print('TEST ID', index_order_nested[index_order_idx][0])

        print('-----Compile model-----')
        model = Transformer()
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 1e-3) #, momentum = 0.99)

        print('-----Train-----')
        path_saver = model_folder + pkl(id_, n_layers, n_heads, fold, d_model)

        print('path_saver: ', path_saver)

        metric_best, ep_best = 0, -1
        time_train = 0

        for epoch in range(1, epochs + 1):
            print("Epoch:", epoch)
            time_train_f = time.time()
            ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
            ys_val, loss_val_list, metrics_val, _prob, _binary = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

            train_fold_metrics_list.append(metrics_train)
            val_fold_metrics_list.append(metrics_val)

            metrics_ep_avg = sum(metrics_val[:4])/4
            if metrics_ep_avg > metric_best: 
                metric_best, ep_best = metrics_ep_avg, epoch
                torch.save(model.eval().state_dict(), path_saver)
                #torch.save(model.state_dict(), path_saver)
            print("Fold time: {:6.2f} mins.".format( round( (time.time()-time_train_f)/60, 2) ))
        print("best_epoch:", ep_best)
        print('-----Optimization Finished!-----')
        del train_pep_inputs, train_hla_inputs, train_labels, train_loader
        del val_pep_inputs, val_hla_inputs, val_labels, val_loader
        gc.collect()
        
print("Total training time: {:6.2f} mins.".format( round( (time.time()-time_train_c)/60, 2) ))


  0%|                                                                                                              | 0/4 [00:00<?, ?it/s]

=====Inner Fold-16=====
-----Generate data loader-----
16
prepare training data:
make data 1 done
make data 2 done
make data 3 done
make data done
hla done
pep done
labels done
TRAIN ID: [1, 2, 3] 1807901
prepare training data done.
shape pep inputs: torch.Size([1807901, 14, 20])
shape hla inputs: torch.Size([1807901, 34, 20])
shape labels: torch.Size([1807901])
prepare val data:
VAL ID: 0 603729
prepare val data done.
shape pep inputs: torch.Size([603729, 14, 20])
shape hla inputs: torch.Size([603729, 34, 20])
shape labels: torch.Size([603729])
TEST ID 4
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL16.pkl
Epoch: 1
Fold-16****Train (Ep avg): Epoch-1/25 | Loss = 0.2340 | Time = 0.7861 sec
tn = 817742, fp = 87732, fn = 82571, tp = 819856
y_pred: 0 = 900313 | 1 = 907588
y_true: 0 = 905474 | 1 = 902427
auc=0.9666|sensitivity=0.9085|specificity=0.9031|acc=0.905

Fold-16****Train (Ep avg): Epoch-13/25 | Loss = 0.1612 | Time = 0.8970 sec
tn = 846931, fp = 58543, fn = 54099, tp = 848328
y_pred: 0 = 901030 | 1 = 906871
y_true: 0 = 905474 | 1 = 902427
auc=0.9837|sensitivity=0.9401|specificity=0.9353|acc=0.9377|mcc=0.8754
precision=0.9354|recall=0.9401|f1=0.9377|aupr=0.9825
Fold-16 ****Test  Epoch-13/25: Loss = 0.196186
tn = 281093, fp = 20410, fn = 24476, tp = 277750
y_pred: 0 = 305569 | 1 = 298160
y_true: 0 = 301503 | 1 = 302226
auc=0.9772|sensitivity=0.9190|specificity=0.9323|acc=0.9257|mcc=0.8514
precision=0.9315|recall=0.9190|f1=0.9252|aupr=0.9766
Fold time:   0.70 mins.
Epoch: 14
Fold-16****Train (Ep avg): Epoch-14/25 | Loss = 0.1597 | Time = 0.9030 sec
tn = 847509, fp = 57965, fn = 53484, tp = 848943
y_pred: 0 = 900993 | 1 = 906908
y_true: 0 = 905474 | 1 = 902427
auc=0.9840|sensitivity=0.9407|specificity=0.9360|acc=0.9384|mcc=0.8767
precision=0.9361|recall=0.9407|f1=0.9384|aupr=0.9828
Fold-16 ****Test  Epoch-14/25: Loss = 0.197233
tn = 282033

 25%|█████████████████████████                                                                           | 1/4 [19:16<57:49, 1156.38s/it]

=====Inner Fold-17=====
-----Generate data loader-----
17
prepare training data:
make data 0 done
make data 2 done
make data 3 done
make data done
hla done
pep done
labels done
TRAIN ID: [0, 2, 3] 1809399
prepare training data done.
shape pep inputs: torch.Size([1809399, 14, 20])
shape hla inputs: torch.Size([1809399, 34, 20])
shape labels: torch.Size([1809399])
prepare val data:
VAL ID: 1 602231
prepare val data done.
shape pep inputs: torch.Size([602231, 14, 20])
shape hla inputs: torch.Size([602231, 34, 20])
shape labels: torch.Size([602231])
TEST ID 4
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL17.pkl
Epoch: 1
Fold-17****Train (Ep avg): Epoch-1/25 | Loss = 0.2332 | Time = 0.9249 sec
tn = 817499, fp = 87594, fn = 82188, tp = 822118
y_pred: 0 = 899687 | 1 = 909712
y_true: 0 = 905093 | 1 = 904306
auc=0.9668|sensitivity=0.9091|specificity=0.9032|acc=0.906

...

 75%|███████████████████████████████████████████████████████████████████████████                         | 3/4 [58:57<19:47, 1187.69s/it]

=====Inner Fold-19=====
-----Generate data loader-----
19
prepare training data:
make data 0 done
make data 1 done
make data 2 done
make data done
hla done
pep done
labels done
TRAIN ID: [0, 1, 2] 1809316
prepare training data done.
shape pep inputs: torch.Size([1809316, 14, 20])
shape hla inputs: torch.Size([1809316, 34, 20])
shape labels: torch.Size([1809316])
prepare val data:
VAL ID: 3 602314
prepare val data done.
shape pep inputs: torch.Size([602314, 14, 20])
shape hla inputs: torch.Size([602314, 34, 20])
shape labels: torch.Size([602314])
TEST ID 4
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL19.pkl
Epoch: 1
Fold-19****Train (Ep avg): Epoch-1/25 | Loss = 0.2332 | Time = 0.9551 sec
tn = 818265, fp = 87213, fn = 82315, tp = 821523
y_pred: 0 = 900580 | 1 = 908736
y_true: 0 = 905478 | 1 = 903838
auc=0.9668|sensitivity=0.9089|specificity=0.9037|acc=0.906

Fold-19****Train (Ep avg): Epoch-13/25 | Loss = 0.1615 | Time = 1.0756 sec
tn = 847260, fp = 58218, fn = 54858, tp = 848980
y_pred: 0 = 902118 | 1 = 907198
y_true: 0 = 905478 | 1 = 903838
auc=0.9837|sensitivity=0.9393|specificity=0.9357|acc=0.9375|mcc=0.8750
precision=0.9358|recall=0.9393|f1=0.9376|aupr=0.9825
Fold-19 ****Test  Epoch-13/25: Loss = 0.200223
tn = 280613, fp = 20886, fn = 25975, tp = 274840
y_pred: 0 = 306588 | 1 = 295726
y_true: 0 = 301499 | 1 = 300815
auc=0.9761|sensitivity=0.9137|specificity=0.9307|acc=0.9222|mcc=0.8445
precision=0.9294|recall=0.9137|f1=0.9214|aupr=0.9756
Fold time:   0.83 mins.
Epoch: 14
Fold-19****Train (Ep avg): Epoch-14/25 | Loss = 0.1600 | Time = 1.0035 sec
tn = 847657, fp = 57821, fn = 54126, tp = 849712
y_pred: 0 = 901783 | 1 = 907533
y_true: 0 = 905478 | 1 = 903838
auc=0.9839|sensitivity=0.9401|specificity=0.9361|acc=0.9381|mcc=0.8763
precision=0.9363|recall=0.9401|f1=0.9382|aupr=0.9827
Fold-19 ****Test  Epoch-14/25: Loss = 0.199292
tn = 279927

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [1:18:50<00:00, 1182.52s/it]

Total training time:  78.83 mins.


  0%|                                                                                            | 0/20 [00:00<?, ?it/s]

=====Inner Fold-0=====
-----Generate data loader-----
0
prepare training data:
make data 2 done
make data 3 done
make data 4 done
make data done
hla done
pep done
labels done
TRAIN ID: [2, 3, 4] 1811042
prepare training data done.
shape pep inputs: torch.Size([1811042, 14, 20])
shape hla inputs: torch.Size([1811042, 34, 20])
shape labels: torch.Size([1811042])
prepare val data:
VAL ID: 1 602231
prepare val data done.
shape pep inputs: torch.Size([602231, 14, 20])
shape hla inputs: torch.Size([602231, 34, 20])
shape labels: torch.Size([602231])
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL0.pkl
Epoch: 1
Fold-0****Train (Ep avg): Epoch-1/25 | Loss = 0.2327 | Time = 1.1027 sec
tn = 817241, fp = 87873, fn = 82088, tp = 823840
y_pred: 0 = 899329 | 1 = 911713
y_true: 0 = 905114 | 1 = 905928
auc=0.9669|sensitivity=0.9094|specificity=0.9029|acc=0.9062|mc

Fold-0****Train (Ep avg): Epoch-13/25 | Loss = 0.1626 | Time = 1.2058 sec
tn = 846078, fp = 59036, fn = 54537, tp = 851391
y_pred: 0 = 900615 | 1 = 910427
y_true: 0 = 905114 | 1 = 905928
auc=0.9834|sensitivity=0.9398|specificity=0.9348|acc=0.9373|mcc=0.8746
precision=0.9352|recall=0.9398|f1=0.9375|aupr=0.9823
Fold-0 ****Test  Epoch-13/25: Loss = 0.200709
tn = 281992, fp = 19892, fn = 26116, tp = 274231
y_pred: 0 = 308108 | 1 = 294123
y_true: 0 = 301884 | 1 = 300347
auc=0.9765|sensitivity=0.9130|specificity=0.9341|acc=0.9236|mcc=0.8474
precision=0.9324|recall=0.9130|f1=0.9226|aupr=0.9761
Fold time:   0.90 mins.
Epoch: 14
Fold-0****Train (Ep avg): Epoch-14/25 | Loss = 0.1613 | Time = 1.1838 sec
tn = 846449, fp = 58665, fn = 53987, tp = 851941
y_pred: 0 = 900436 | 1 = 910606
y_true: 0 = 905114 | 1 = 905928
auc=0.9837|sensitivity=0.9404|specificity=0.9352|acc=0.9378|mcc=0.8756
precision=0.9356|recall=0.9404|f1=0.9380|aupr=0.9825
Fold-0 ****Test  Epoch-14/25: Loss = 0.202169
tn = 281284, fp

  5%|████                                                                            | 1/20 [24:56<7:53:52, 1496.42s/it]

y_true: 0 = 301884 | 1 = 300347
auc=0.9763|sensitivity=0.9151|specificity=0.9321|acc=0.9237|mcc=0.8474
precision=0.9306|recall=0.9151|f1=0.9228|aupr=0.9760
Fold time:   0.88 mins.
best_epoch: 24
-----Optimization Finished!-----
=====Inner Fold-1=====
-----Generate data loader-----
1
prepare training data:
make data 1 done
make data 3 done
make data 4 done
make data done
hla done
pep done
labels done
TRAIN ID: [1, 3, 4] 1809917
prepare training data done.
shape pep inputs: torch.Size([1809917, 14, 20])
shape hla inputs: torch.Size([1809917, 34, 20])
shape labels: torch.Size([1809917])
prepare val data:
VAL ID: 2 603356
prepare val data done.
shape pep inputs: torch.Size([603356, 14, 20])
shape hla inputs: torch.Size([603356, 34, 20])
shape labels: torch.Size([603356])
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL1.pkl
Epoch: 1
Fold-1****Train (Ep 

...

y_true: 0 = 302091 | 1 = 301265
auc=0.9757|sensitivity=0.9088|specificity=0.9365|acc=0.9227|mcc=0.8457
precision=0.9345|recall=0.9088|f1=0.9215|aupr=0.9751
Fold time:   0.84 mins.
best_epoch: 18
-----Optimization Finished!-----
=====Inner Fold-15=====
-----Generate data loader-----
15
prepare training data:
make data 0 done
make data 1 done
make data 2 done
make data done
hla done
pep done
labels done
TRAIN ID: [0, 1, 2] 1809316
prepare training data done.
shape pep inputs: torch.Size([1809316, 14, 20])
shape hla inputs: torch.Size([1809316, 34, 20])
shape labels: torch.Size([1809316])
prepare val data:
VAL ID: 4 605372
prepare val data done.
shape pep inputs: torch.Size([605372, 14, 20])
shape hla inputs: torch.Size([605372, 34, 20])
shape labels: torch.Size([605372])
TEST ID 3
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL15.pkl
Epoch: 1
Fold-15****Train 

Fold-15 ****Test  Epoch-12/25: Loss = 0.196175
tn = 278472, fp = 23052, fn = 22766, tp = 281082
y_pred: 0 = 301238 | 1 = 304134
y_true: 0 = 301524 | 1 = 303848
auc=0.9769|sensitivity=0.9251|specificity=0.9235|acc=0.9243|mcc=0.8486
precision=0.9242|recall=0.9251|f1=0.9246|aupr=0.9765
Fold time:   0.83 mins.
Epoch: 13
Fold-15****Train (Ep avg): Epoch-13/25 | Loss = 0.1621 | Time = 1.1185 sec
tn = 846980, fp = 58498, fn = 54651, tp = 849187
y_pred: 0 = 901631 | 1 = 907685
y_true: 0 = 905478 | 1 = 903838
auc=0.9835|sensitivity=0.9395|specificity=0.9354|acc=0.9375|mcc=0.8749
precision=0.9356|recall=0.9395|f1=0.9375|aupr=0.9823
Fold-15 ****Test  Epoch-13/25: Loss = 0.195504
tn = 279683, fp = 21841, fn = 23901, tp = 279947
y_pred: 0 = 303584 | 1 = 301788
y_true: 0 = 301524 | 1 = 303848
auc=0.9771|sensitivity=0.9213|specificity=0.9276|acc=0.9244|mcc=0.8489
precision=0.9276|recall=0.9213|f1=0.9245|aupr=0.9768
Fold time:   0.84 mins.
Epoch: 14
Fold-15****Train (Ep avg): Epoch-14/25 | Loss = 0.16

tn = 280415, fp = 21109, fn = 24937, tp = 278911
y_pred: 0 = 305352 | 1 = 300020
y_true: 0 = 301524 | 1 = 303848
auc=0.9767|sensitivity=0.9179|specificity=0.9300|acc=0.9239|mcc=0.8479
precision=0.9296|recall=0.9179|f1=0.9237|aupr=0.9765
Fold time:   0.84 mins.
best_epoch: 15
-----Optimization Finished!-----


 80%|█████████████████████████████████████████████████████████████▌               | 16/20 [6:46:15<1:38:14, 1473.63s/it]

=====Inner Fold-16=====
-----Generate data loader-----
16
prepare training data:
make data 1 done
make data 2 done
make data 3 done
make data done
hla done
pep done
labels done
TRAIN ID: [1, 2, 3] 1807901
prepare training data done.
shape pep inputs: torch.Size([1807901, 14, 20])
shape hla inputs: torch.Size([1807901, 34, 20])
shape labels: torch.Size([1807901])
prepare val data:
VAL ID: 0 603729
prepare val data done.
shape pep inputs: torch.Size([603729, 14, 20])
shape hla inputs: torch.Size([603729, 34, 20])
shape labels: torch.Size([603729])
TEST ID 4
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL16.pkl
Epoch: 1
Fold-16****Train (Ep avg): Epoch-1/25 | Loss = 0.2329 | Time = 1.1277 sec
tn = 817761, fp = 87713, fn = 82174, tp = 820253
y_pred: 0 = 899935 | 1 = 907966
y_true: 0 = 905474 | 1 = 902427
auc=0.9669|sensitivity=0.9089|specificity=0.9031|acc=0.906